In [ ]:
import torch

DEVICE = "cuda:1"
torch.cuda.set_device(DEVICE)

from FooocusSDXLInpaintAllInOnePipeline import FooocusSDXLInpaintPipeline

pipe = FooocusSDXLInpaintPipeline.from_pretrained(
    "frankjoshua/juggernautXL_v8Rundiffusion",
    torch_dtype=torch.float16,
    use_safetensors=True,
).to(DEVICE)

In [ ]:
lora_config = [
    {
        "model_path": f"lora/xxx",
        "scale": 1,
        "for_raw_unet": False,
        "for_fooocus_unet": True,
    },
]

pipe.preload_fooocus_unet(
    fooocus_model_path="./models/fooocus_inpaint/inpaint_v26.fooocus.patch",
    lora_configs=lora_config,
    add_double_sa=False,
)

In [ ]:
from diffusers.utils import load_image
from PIL import Image


img_url = f"data/1_1.jpg"
mask_url = f"data/1_2.jpg"

init_image = load_image(img_url).convert("RGB")
mask_image = load_image(mask_url).convert("RGB")

prompt = ""
negative_prompt = ""

In [ ]:
# Infer！
image = pipe(
    isf_global_time=20,
    isf_global_ia=1,
    decompose_prefix_prompt="a photo of a sks",
    sks_decompose_words=[""],
    fooocus_model_head_path="./models/fooocus_inpaint/fooocus_inpaint_head.pth",
    fooocus_model_head_upscale_path="./models/upscale_models/fooocus_upscaler_s409985e5.bin",
    pag_scale=1,
    guidance_scale=4,
    ref_image_type="no", 
    double_sa_alpha=1,
    save_self_attn=False,
    save_cross_attn=False,
    fooocus_time=0.8,
    inpaint_respective_field=0.5, 
    sharpness=1, 
    adm_scaler_positive=1.5, 
    adm_scaler_negative=0.8,  
    adm_scaler_end=0.3,
    seed=42,
    image=init_image,
    mask_image=mask_image,
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    strength=1,
)
image.resize((512, 512))
image